In [1]:
from netCDF4 import Dataset
from numpy import dtype
import numpy as np

#-----------------
# read netCDF file
#-----------------

# open a netCDF file to read
filename = "JRA/1973/anl_p125.034_vgrd.1973010100_1973013118.nc"
ncin = Dataset(filename, 'r', format='NETCDF4')

In [2]:
lat=ncin['g0_lat_2'][:]
lon=ncin['g0_lon_3'][:]
leve=ncin['lv_ISBL1']
leve, lat.shape, lon.shape

(<class 'netCDF4._netCDF4.Variable'>
 int32 lv_ISBL1(lv_ISBL1)
     units: hPa
     long_name: isobaric level
 unlimited dimensions: 
 current shape = (37,)
 filling on, default _FillValue of -2147483647 used, (145,), (288,))

In [3]:
ncin.variables

OrderedDict([('VGRD_GDS0_ISBL', <class 'netCDF4._netCDF4.Variable'>
              float32 VGRD_GDS0_ISBL(initial_time0_hours, lv_ISBL1, g0_lat_2, g0_lon_3)
                  forecast_time_units: hours
                  forecast_time: 0
                  parameter_number: 34
                  parameter_table_version: 200
                  gds_grid_type: 0
                  level_indicator: 100
                  _FillValue: 1e+20
                  units: m/s
                  long_name: v-component of wind
                  center: Japanese Meteorological Agency - Tokyo (RSMC)
                  sub_center: 241
              unlimited dimensions: 
              current shape = (124, 37, 145, 288)
              filling on),
             ('initial_time0_hours', <class 'netCDF4._netCDF4.Variable'>
              float64 initial_time0_hours(initial_time0_hours)
                  units: hours since 1800-01-01 00:00
                  long_name: initial time
              unlimited dimensions: 
 

In [4]:
# np.where(lat==30), np.where(lat==-52.5), np.where(lon==16.25), np.where(lon==137.5)

In [5]:
# check netCDF file format
#print(ncin.file_format)

# get axis data
#print(ncin.dimensions.keys())
#print(ncin.dimensions['time'])

tin = ncin.variables['initial_time0_hours']
lev = ncin.variables['lv_ISBL1']
latitude = ncin.variables['g0_lat_2']
longitude = ncin.variables['g0_lon_3']
# latitude = ncin.variables['g0_lat_2'][48:114]
# longitude = ncin.variables['g0_lon_3'][13:110]

# get length of axis data
#ntime = len(tin)
nlev = len(lev)
ntime = len(tin)/len(tin)
nlat = len(latitude)
nlon = len(longitude)

In [6]:
longitude.shape, latitude.shape, tin.shape, lev.shape, nlev, ntime, nlat, nlon

((288,), (145,), (124,), (37,), 37, 1.0, 145, 288)

In [7]:
# read data
vin = ncin.variables['VGRD_GDS0_ISBL'][:,:,:,:]
# vin = ncin.variables['HGT_GDS0_ISBL'][:,:,48:114,13:110]
vin.shape

(124, 37, 145, 288)

In [8]:
#rata per bulan

vin = np.nanmean(np.reshape(vin,(124,1,37,145,288)), axis=0)
# vin = np.nanmean(np.reshape(vin,(112,1,37,66,97)), axis=0)
vin.shape

(1, 37, 145, 288)

In [9]:
#------------------
# write netCDF file
#------------------

# open a netCDF file to write
ncout = Dataset('JRA/vwnd/vwnd197301.nc', 'w', format='NETCDF4')

# define axis size
ncout.createDimension('time', ntime)  # unlimited
ncout.createDimension('lv_ISBL1',nlev)
ncout.createDimension('g0_lat_2', nlat)
ncout.createDimension('g0_lon_3', nlon)

<class 'netCDF4._netCDF4.Dimension'>: name = 'g0_lon_3', size = 288

In [10]:
vin.shape

(1, 37, 145, 288)

In [11]:
# create time axis
time = ncout.createVariable('time', dtype('double').char, ('time',))
time.long_name = 'initial time'
time.units = 'hours since 1973-01-01 00:00:0.0'
time.calendar = 'standard'
time.axis = 'T'

#creat level axis
level = ncout.createVariable('lv_ISBL1', dtype('double').char, ('lv_ISBL1',))
level.long_name = 'isobaric level'
level.units = 'hPa'
level.axis= 'Z'

# create latitude axis
lat = ncout.createVariable('g0_lat_2', dtype('double').char, ('g0_lat_2'))
lat.standard_name = 'latitude'
lat.long_name = 'latitude'
lat.units = 'degrees_north'
lat.axis = 'Y'

# create longitude axis
lon = ncout.createVariable('g0_lon_3', dtype('double').char, ('g0_lon_3'))
lon.standard_name = 'longitude'
lon.long_name = 'longitude'
lon.units = 'degrees_east'
lon.axis = 'X'

# create variable array
vout = ncout.createVariable('VGRD_GDS0_ISBL', dtype('double').char, ('time', 'lv_ISBL1', 'g0_lat_2', 'g0_lon_3'))
vout.long_name = 'v-component of wind'
vout.units = 'm/s'

In [12]:
# copy axis from original dataset
level[:] = lev[:]
time[:] = ntime
lon[:] = longitude[:]
lat[:] = latitude[:]
vout[:] = vin[:]

# close files
ncin.close()
ncout.close()

In [13]:
test_filename = "JRA/vwnd/vwnd197301.nc"
testfile = Dataset(test_filename, 'r', format='NETCDF4')

In [14]:
testfile.variables

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
              float64 time(time)
                  long_name: initial time
                  units: hours since 1973-01-01 00:00:0.0
                  calendar: standard
                  axis: T
              unlimited dimensions: 
              current shape = (1,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('lv_ISBL1', <class 'netCDF4._netCDF4.Variable'>
              float64 lv_ISBL1(lv_ISBL1)
                  long_name: isobaric level
                  units: hPa
                  axis: Z
              unlimited dimensions: 
              current shape = (37,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('g0_lat_2', <class 'netCDF4._netCDF4.Variable'>
              float64 g0_lat_2(g0_lat_2)
                  standard_name: latitude
                  long_name: latitude
                  units: degrees_north
                  axis:

In [15]:
testfile['VGRD_GDS0_ISBL']

<class 'netCDF4._netCDF4.Variable'>
float64 VGRD_GDS0_ISBL(time, lv_ISBL1, g0_lat_2, g0_lon_3)
    long_name: v-component of wind
    units: m/s
unlimited dimensions: 
current shape = (1, 37, 145, 288)
filling on, default _FillValue of 9.969209968386869e+36 used